In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from finta import TA
import mplfinance as mpf
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px
import webbrowser
from scipy.signal import find_peaks
import dash
from dash_table import DataTable
import dash_bootstrap_components as dbc
from dash import dcc, html

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8504\2343702631.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8504\2343702631.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8504\2343702631.py:16: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  from dash_table import DataTable


In [8]:
# Đọc dữ liệu từ các tệp Excel
VN2018 = pd.read_excel("2018-Vietnam (1).xlsx")
VN2019 = pd.read_excel("2019-Vietnam (1).xlsx")
VN2020 = pd.read_excel("2020-Vietnam (1).xlsx")
VN2021 = pd.read_excel("2021-Vietnam (1).xlsx")
VN2022 = pd.read_excel("2022-Vietnam (1).xlsx")
# Nối dữ liệu
inf = pd.concat([VN2018, VN2019, VN2020, VN2021, VN2022], ignore_index=True)
# Tiền xử lý dữ liệu
inf = inf.iloc[7:].reset_index(drop=True)
inf.columns = inf.iloc[0]
inf = inf[1:]
inf.reset_index(drop=True, inplace=True)
inf = pd.DataFrame(inf)
# Danh sách các cột không muốn thay đổi
excluded_columns = ["STT", "Mã", "Tên công ty", "Sàn", "Ngành ICB - cấp 1", "Ngành ICB - cấp 2", "Ngành ICB - cấp 3", "Ngành ICB - cấp 4"]
# Kiểm tra và loại bỏ khoảng trắng từ tên cột (ngoại trừ các cột được liệt kê trong excluded_columns)
inf.columns = inf.columns.where(inf.columns.isin(excluded_columns), inf.columns.str.extract(r'\s*([^\n]+)\n', expand=False))
inf.index.name = "year"
# Lấy dữ liệu của mã chứng khoán "BCM" từ DataFrame "inf"
bcm_data = inf[inf['Mã'] == 'BCM'].copy()
# In ra DataFrame chứa dữ liệu của mã "BCM"
bcm_data


,STT,Mã,Tên công ty,Sàn,Ngành ICB - cấp 1,Ngành ICB - cấp 2,Ngành ICB - cấp 3,Ngành ICB - cấp 4,Quý,Năm,...,TM. Chi phí tài chính khác,TM. Chi phí sản xuất theo yếu tố,"TM. Chi phí nguyên liệu, vật liệu",TM. Chi phí nhân công,TM.Chi phí khấu hao tài sản cố định,TM. Chi phí dịch vụ mua ngoài,TM. Chi phí khác bằng tiền,BCTCKH. Doanh thu kế hoạch,BCTCKH. Tổng lợi nhuận kế toán trước thuế,BCTCKH. Lợi nhuận sau thuế thu nhập doanh nghiệp
year,,,,,,,,,,,,,,,,,,,,,
1,2,BCM,Becamex IDC,HOSE,Tài chính,Bất động sản,Bất động sản,Bất động sản,5,2018,...,3543121946,9062957842701,1451720418454,1039031680467,223031443148,5870329556428,478844744204,8998000000000,2897000000000,2613000000000
1673,2,BCM,Becamex IDC,HOSE,Tài chính,Bất động sản,Bất động sản,Bất động sản,5,2019,...,11724673883,7350155810474,1056126016274,1141883252530,225844734008,4324074628654,602227179008,6016000000000,1152000000000,931000000000
3345,2,BCM,Becamex IDC,HOSE,Tài chính,Bất động sản,Bất động sản,Bất động sản,5,2020,...,17202053049,6514457410506,1199685932724,969524854449,238000023163,3475601230334,631645369836,8900000000000,2700000000000,2300000000000
5017,2,BCM,Becamex IDC,HOSE,Tài chính,Bất động sản,Bất động sản,Bất động sản,5,2021,...,0,0,0,0,0,0,0,9680000000000,3256000000000,2888000000000
6689,2,BCM,Becamex IDC,HOSE,Tài chính,Bất động sản,Bất động sản,Bất động sản,5,2022,...,194442752,5341905292230,573310732794,865990973170,189713347046,3386322904665,326567334555,9460000000000,2628000000000,2263000000000


In [3]:
#XỬ LÝ FILE GIÁ
# Đọc DataFrame từ tệp Excel vào biến filename
filename = pd.read_excel("Price-Vol VN 2015-2023.xlsx", sheet_name='Price')
# Sử dụng DataFrame filename trực tiếp, không cần đọc lại từ tệp Excel
df = filename.fillna(0)
df.Code = df.Code.str.replace('VT:', '').str.replace('(P)', '')
bcm_data = df[df['Code'] == 'BCM']
df2 = bcm_data.T
df3 = df2.reset_index()
df3.columns = ['Date','Close']
# Loại bỏ các dòng có ít nhất một giá trị NaN
df3 = df3.iloc[3:].reset_index(drop = True)
df3['Date'] = pd.to_datetime(df3['Date'])
df3

,Date,Close
0,2015-01-01,0
1,2015-01-02,0.0
2,2015-01-05,0.0
3,2015-01-06,0.0
4,2015-01-07,0.0
...,...,...
2303,2023-10-31,58600.0
2304,2023-11-01,58000.0
2305,2023-11-02,58800.0
2306,2023-11-03,58500.0


In [4]:
#XỬ LÝ FILE VOLUME
volume = pd.read_excel("Price-Vol VN 2015-2023.xlsx", sheet_name='Volume')
vl = volume.fillna(0)
vl.Code = vl.Code.str.replace('VT:', '').str.replace('(VO)', '')
vl1 = vl[vl['Code'] == 'BCM']
vl3 = vl1.T
vl4 = vl3.reset_index()
vl4 = vl4.iloc[3:].reset_index(drop=True)
vl4.columns = ['Date', 'Vol']
# Add the numeric 'Close' values to vl4
vl4['Close'] = df3['Close']*1
# Print or use vl4 as needed
print(vl4)

                     Date    Vol    Close
0     2015-01-01 00:00:00      0        0
1     2015-01-02 00:00:00    0.0      0.0
2     2015-01-05 00:00:00    0.0      0.0
3     2015-01-06 00:00:00    0.0      0.0
4     2015-01-07 00:00:00    0.0      0.0
...                   ...    ...      ...
2303  2023-10-31 00:00:00  206.0  58600.0
2304  2023-11-01 00:00:00  166.4  58000.0
2305  2023-11-02 00:00:00  265.2  58800.0
2306  2023-11-03 00:00:00  216.2  58500.0
2307  2023-11-06 00:00:00  263.6  59400.0

[2308 rows x 3 columns]


In [5]:

#XỬ LÍ FILE THÔNG TIN
volume = pd.read_excel("Price-Vol VN 2015-2023.xlsx", sheet_name='Info')
df_info = volume
df_info.Symbol = df_info.Symbol.str.replace('VT:', '')
df_info = df_info[df_info['Symbol'] == 'BCM']
print(df_info)

                                       Name Symbol     RIC Start Date  Hist.  \
1357  INVESTMENT AND INDUSTRIAL DEVELOPMENT    BCM  BCM.HM  2/21/2018   2018   

      Category     Exchange   Market         Currency  \
1357  Equities  Ho Chi Minh  Vietnam  Vietnamese Dong   

                                   Sector  \
1357  Real Estate Investment and Services   

                                  Full Name Activity  
1357  Investment and Industrial Development   Active  


In [9]:
# Tạo ứng dụng Dash
app = dash.Dash(__name__)

#TẠO BIỂU ĐỒ TĂNG TRƯỞNG DOANH THU THUẦN
# Tính toán tăng trưởng
revenues = bcm_data.copy()['KQKD. Doanh thu thuần'].tolist()
growth_rates = [(revenues[i] - revenues[i-1]) / abs(revenues[i-1]) * 100 for i in range(1, len(revenues))]

# Chuẩn bị dữ liệu cho biểu đồ đường tăng trưởng
years = bcm_data.iloc[1:]['Năm'].tolist()
years = [int(year) for year in years]

# Chuẩn bị dữ liệu cho biểu đồ cột doanh thu thuần
revenue_values = bcm_data.iloc[1:]['KQKD. Doanh thu thuần'].tolist()

# Thêm cột mới 'TangTruongPercent' vào bcm_data
bcm_data['TangTruongPercent'] = [None] + growth_rates

fig1 = go.Figure()

# Tạo biểu đồ đường tăng trưởng
fig1.add_trace(
    go.Scatter(x=years, y=growth_rates, mode='lines+markers', name='Tăng trưởng', yaxis='y2', line=dict(color='red'))
)

# Tạo biểu đồ cột doanh thu thuần
fig1.add_trace(
    go.Bar(x=years, y=revenue_values, name='Doanh thu thuần', marker_color='skyblue')
)

fig1.update_layout(yaxis2=dict(title='Tăng trưởng (%)', overlaying='y', side='right'))

# Cấu hình giao diện tương tá
fig1.update_layout(
    title='Tăng trưởng doanh thu (Năm)',
    xaxis=dict(title='Năm', tickmode='linear', tick0=years[0], dtick=1),
    yaxis=dict(title='Doanh thu thuần (Tỷ VND)'),
    legend=dict(x=0, y=1),
    hovermode='x'
)

#BIỂU ĐỒ DOANH THU-LN GỘP-LNST
# Tính toán lợi nhuận sau thuế của công ty mẹ
bcm_data['LoiNhuanSauThue'] = bcm_data['KQKD. Lợi nhuận sau thuế thu nhập doanh nghiệp'] - bcm_data['KQKD. Lợi ích của cổ đông thiểu số']

# Dữ liệu doanh thu thuần
revenues = bcm_data['KQKD. Doanh thu thuần'].tolist()

# Lợi nhuận gộp và lợi nhuận sau thuế
profit3 = bcm_data['KQKD. Lợi nhuận gộp về bán hàng và cung cấp dịch vụ'].tolist()
net_profit = bcm_data['LoiNhuanSauThue'].tolist()

# Năm
years = bcm_data['Năm'].tolist()

# Xây dựng grouped bar chart
bar_width = 0.28
index = np.arange(len(years))

# Tạo biểu đồ cột cho doanh thu, lợi nhuận gộp và lợi nhuận sau thuế
fig2 = go.Figure()

fig2.add_trace(go.Bar(x=index - bar_width, y=revenues, name='Doanh thu'))
fig2.add_trace(go.Bar(x=index, y=profit3, name='Lợi nhuận gộp'))
fig2.add_trace(go.Bar(x=index + bar_width, y=net_profit, name='Lợi nhuận sau thuế'))

# Tạo layout
fig2.update_layout(
    title='[BCM] Doanh thu - LN gộp - LNST theo năm',
    xaxis=dict(title='Năm tài chính'),
    yaxis=dict(title='Số tiền (đơn vị tỷ đồng)'),
    xaxis_tickvals=index,
    xaxis_ticktext=years,
    barmode='group',
    legend=dict(x=1.02, y=1.0),
)
#BIỂU ĐỒ CƠ CẤU TÀI SẢN
 # Dữ liệu năm
years = bcm_data['Năm'].tolist()

# Dữ liệu tài sản dài hạn và tài sản ngắn hạn
asset = bcm_data['CĐKT. TÀI SẢN DÀI HẠN'].tolist()
asset2 = bcm_data['CĐKT. TÀI SẢN NGẮN HẠN'].tolist()

# Tạo DataFrame chỉ với tài sản dài hạn và tài sản ngắn hạn
df = pd.DataFrame({
    'Năm': years * 2,
    'Loại Tài Sản': ['Tài sản dài hạn'] * len(years) + ['Tài sản ngắn hạn'] * len(years),
    'Giá trị Tài Sản (tỷ đồng)': asset + asset2
})

# Vẽ biểu đồ
fig3 = px.bar(df, x='Năm', y='Giá trị Tài Sản (tỷ đồng)', color='Loại Tài Sản',
             labels={'Giá trị Tài Sản (tỷ đồng)': 'Giá trị Tài Sản'},
             title='[BCM] CƠ CẤU TÀI SẢN THEO NĂM')
#CƠ CẤU TÀI SẢN DÀI HẠN
# Dữ liệu năm
years = bcm_data['Năm'].tolist()

# Dữ liệu tài sản dài hạn và tài sản ngắn hạn
asset = bcm_data['CĐKT. TÀI SẢN DÀI HẠN'].tolist()
receivables = bcm_data['CĐKT. Phải thu dài hạn'].tolist()
Lassets = bcm_data['CĐKT. Tài sản cố định'].tolist()
Lassets2 = bcm_data['CĐKT. Tài sản dở dang dài hạn'].tolist()
Lassets3 = bcm_data['CĐKT. Đầu tư dài hạn'].tolist()
Lassets4 = bcm_data['CĐKT. Tài sản dài hạn khác'].tolist()


# Tạo DataFrame cho Plotly Express
df = pd.DataFrame({
    'Năm': years * 5,
    'Loại Tài Sản': (['Tài sản cố định'] * len(years) + ['Tài sản dở dang dài hạn'] * len(years) +
                     ['Đầu tư dài hạn'] * len(years) + ['Tài sản dài hạn khác'] * len(years) +
                     ['Phải thu dài hạn'] * len(years)),
    'Giá trị Tài Sản (tỷ đồng)': (Lassets + Lassets2 + Lassets3 + Lassets4 + receivables)
})

# Vẽ biểu đồ cột ghép
fig4 = px.bar(df, x='Năm', y='Giá trị Tài Sản (tỷ đồng)', color='Loại Tài Sản',
             title='Cơ cấu tài sản dài hạn của BCM theo năm',
             labels={'Giá trị Tài Sản (tỷ đồng)': 'Giá trị Tài Sản'},
             barmode='group')  

#CƠ CẤU TÀI SẢN NGẮN HẠN
# Dữ liệu năm
years = bcm_data['Năm'].tolist()

#Các dữ liệu TSNH
asset2 = bcm_data['CĐKT. TÀI SẢN NGẮN HẠN'].tolist()
money = bcm_data['CĐKT. Tiền và tương đương tiền '].tolist()
invest = bcm_data['CĐKT. Đầu tư tài chính ngắn hạn'].tolist()
received = bcm_data['CĐKT. Các khoản phải thu ngắn hạn'].tolist()
assetshort= bcm_data['CĐKT. Hàng tồn kho, ròng'].tolist()
asset_another= bcm_data['CĐKT. Tài sản ngắn hạn khác'].tolist()

# Tạo DataFrame cho Plotly Express
df = pd.DataFrame({
    'Năm': years * 6,
    'Loại Tài Sản Ngắn Hạn': (['Tài sản ngắn hạn'] * len(years) + ['Tiền và tương đương tiền'] * len(years) +
                               ['Đầu tư tài chính ngắn hạn'] * len(years) + ['Các khoản phải thu ngắn hạn'] * len(years) +
                               ['Hàng tồn kho, ròng'] * len(years) + ['Tài sản ngắn hạn khác'] * len(years)),
    'Giá trị Tài Sản (tỷ đồng)': (asset2 + money + invest + received + assetshort + asset_another)
})

# Vẽ biểu đồ cột ghép
fig5 = px.bar(df, x='Năm', y='Giá trị Tài Sản (tỷ đồng)', color='Loại Tài Sản Ngắn Hạn',
             title='Cơ cấu tài sản ngắn hạn của BCM theo năm',
             labels={'Giá trị Tài Sản (tỷ đồng)': 'Giá trị Tài Sản'},
             barmode='group') 

#BIÊN LỢI NHUẬN
# Dữ liệu năm
years = bcm_data['Năm'].tolist()
# Dữ liệu tính toán
interet = bcm_data['KQKD. Lợi nhuận gộp về bán hàng và cung cấp dịch vụ'].tolist()
interet2 = bcm_data['KQKD. Tổng lợi nhuận kế toán trước thuế'].tolist()
expense = bcm_data['KQKD. Trong đó: Chi phí lãi vay']
revenues = bcm_data['KQKD. Doanh thu thuần'].tolist()
profit = bcm_data['KQKD. Lợi nhuận sau thuế thu nhập doanh nghiệp'].tolist()
profit2 = bcm_data['KQKD. Lợi ích của cổ đông thiểu số'].tolist()
# Tính toán lợi nhuận sau thuế của công ty mẹ
net_profit = [p1 - p2 for p1, p2 in zip(profit, profit2)]

# Tính toán biên lợi nhuận gộp
gross_margin = [(i / r) * 100 if r != 0 else 0 for r, i in zip(revenues, interet)]

# Tính toán biên lợi nhuận ròng
net_margin = [(np / r) * 100 if r != 0 else 0 for np, r in zip(net_profit, revenues)]

# Tính toán biên lợi nhuận EBIT
ebit_margin = [((np + e) / r) * 100 if r != 0 else 0 for np, e, r in zip(net_profit, expense, revenues)]

# Tạo DataFrame cho Plotly Express
df = pd.DataFrame({
    'Năm': years * 3,
    'Loại Biên Lợi Nhuận': ['Biên lợi nhuận gộp'] * len(years) + ['Biên lợi nhuận ròng'] * len(years) + ['Biên lợi nhuận EBIT'] * len(years),
    'Phần Trăm': gross_margin + net_margin + ebit_margin
})

# Vẽ biểu đồ
fig6 = px.line(df, x='Năm', y='Phần Trăm', color='Loại Biên Lợi Nhuận',
            labels={'Phần Trăm': 'Phần Trăm (%)'},
            title='[BCM] Biên Lợi Nhuận Theo Năm')
 # Cấu hình định dạng trục x để hiển thị giá trị năm là số nguyên
fig6.update_xaxes(type='category', tickmode='linear')

#KHẢ NĂNG SINH LỜI
#Phân tích khả năng sinh lời của doanh nghiệp sẽ tính các chỉ số như ROS, ROE, ROA
revenues = bcm_data['KQKD. Doanh thu thuần'].tolist()
profit = bcm_data['KQKD. Lợi nhuận sau thuế thu nhập doanh nghiệp'].tolist()
sumprofit = bcm_data ['CĐKT. TỔNG CỘNG TÀI SẢN'].tolist()
equity = bcm_data ['CĐKT. VỐN CHỦ SỞ HỮU'].tolist()
# Tính tỷ suất lợi nhuận ròng (ROS)
ros = [(p / r) * 100 if r != 0 else 0 for p, r in zip(profit, revenues)]

# Tính tỷ suất sinh lời trên tài sản (ROE)
roe = [(p / s) * 100 if s != 0 else 0 for p, s in zip(profit, sumprofit)]

# Tính tỷ suất sinh lời trên vốn chủ sở hữu (ROA)
roa = [(p / e) * 100 if e != 0 else 0 for p, e in zip(profit, equity)]

# Tạo DataFrame cho Plotly Express
df_ratios = pd.DataFrame({
    'Năm': years * 3,
    'Chỉ Số': (['ROS'] * len(years) + ['ROE'] * len(years) + ['ROA'] * len(years)),
    'Tỷ Suất (%)': (ros + roe + roa)
})

# Vẽ biểu đồ đường
fig7 = px.line(df_ratios, x='Năm', y='Tỷ Suất (%)', color='Chỉ Số',
                      title='[BCM] Khả năng sinh lời qua từng năm',
                      labels={'Tỷ Suất (%)': 'Tỷ Suất'})
 # Cấu hình định dạng trục x để hiển thị giá trị năm là số nguyên
fig7.update_xaxes(type='category', tickmode='linear')

#BIỂU ĐỒ DÒNG TIỀN
# Lấy dữ liệu từ DataFrame
currency_flow = bcm_data['LCTT. Lưu chuyển tiền tệ ròng từ các hoạt động sản xuất kinh doanh (TT)'].tolist()
currency_flow2 = bcm_data['LCTT. Lưu chuyển tiền tệ ròng từ hoạt động đầu tư (TT)'].tolist()
currency_flow3 = bcm_data['LCTT. Lưu chuyển tiền tệ từ hoạt động tài chính (TT)'].tolist()
currency_flow5 = bcm_data['LCTT. Tiền và tương đương tiền cuối kỳ (TT)'].tolist()
years = bcm_data['Năm'].tolist()
# Tạo DataFrame cho Plotly Express
df_cash_flow = pd.DataFrame({
    'Năm': years,
    'Lưu chuyển từ sản xuất kinh doanh': currency_flow,
    'Lưu chuyển từ hoạt động đầu tư': currency_flow2,
    'Lưu chuyển từ hoạt động tài chính': currency_flow3,
    'Tiền và tương đương tiền cuối kỳ': currency_flow5
})

# Vẽ biểu đồ cột ghép và đường
fig8 = go.Figure()

# Thêm cột cho từng loại dòng tiền
for column in df_cash_flow.columns[1:]:
    fig8.add_trace(go.Bar(x=df_cash_flow['Năm'], y=df_cash_flow[column], name=column))

# Thêm đường cho 'Tiền và tương đương tiền cuối kỳ'
fig8.add_trace(go.Scatter(x=years, y=currency_flow5, mode='lines+markers',
                          name='LCTT. Tiền và tương đương tiền cuối kỳ (TT)',
                          line=dict(color='red')))

# Cấu hình giao diện tương tác
fig8.update_layout(
    title='[BCM] Biểu đồ Dòng tiền 2018 - 2022',
    xaxis=dict(title='Năm'),
    yaxis=dict(title='Giá trị Tiền (tỷ VNĐ)'),
    barmode='group'  # Đặt barmode thành 'group' để tạo biểu đồ cột ghép
)


#VẼ BIỂU ĐỒ THỂ HIỆN CẤU TRÚC NGUỒN VỐN QUA CÁC NĂM
# Chuyển định dạng dữ liệu để phù hợp với biểu đồ
stacked_data = pd.melt(bcm_data, id_vars=['Năm'], 
                       value_vars=['CĐKT. Phải trả người bán ngắn hạn', 
                                   'CĐKT. Người mua trả tiền trước ngắn hạn', 
                                   'CĐKT. Phải trả nhà cung cấp dài hạn', 
                                   'CĐKT. Vốn góp của chủ sở hữu',
                                   'CĐKT. Thặng dư vốn cổ phần',
                                   'CĐKT.Vốn khác',
                                  'CĐKT. LNST chưa phân phối kỳ này',
                                  'CĐKT. Nợ ngắn hạn',
                                  'CĐKT. Nợ dài hạn',
                                  'CĐKT. Doanh thu chưa thực hiện ngắn hạn',
                                  'CĐKT.Doanh thu chưa thực hiên dài hạn',
                                  'CĐKT. Người mua trả tiền trước dài hạn'],
                       var_name='Loại Nghĩa vụ', value_name='Giá trị')

# Nhóm dữ liệu theo 'Năm' và 'Loại Nghĩa vụ', tính tổng giá trị cho mỗi nhóm
grouped_data = stacked_data.groupby(['Năm', 'Loại Nghĩa vụ'], as_index=False)['Giá trị'].sum()

# Kiểm tra và xử lý nhóm có giá trị bằng 0
zero_value_groups = grouped_data[grouped_data['Giá trị'] == 0]
if not zero_value_groups.empty:
    # Loại bỏ nhóm có giá trị bằng 0 hoặc thay thế giá trị 0 bằng giá trị khác
    grouped_data['Giá trị'] = grouped_data['Giá trị'].replace(0, 1e-6)  # Thay thế giá trị 0 bằng một giá trị rất nhỏ

# Tạo biểu đồ dạng cây (treemap) với tùy chọn màu sắc và giao diện
fig10 = px.treemap(grouped_data, 
                 path=['Năm', 'Loại Nghĩa vụ'], 
                 values='Giá trị',
                 title='[BCM] Cấu trúc Nguồn Vốn qua các Năm',
                 labels={'Giá trị': 'Giá trị (đơn vị)'},
                 color='Giá trị',  # Màu sắc dựa trên giá trị
                 color_continuous_scale='greens')  
#BIỂU ĐỒ GIÁ
# Đoạn mã tính toán các đường MA
df3['MA_50'] = df3['Close'].rolling(window=50).mean()
df3['MA_200'] = df3['Close'].rolling(window=200).mean()

# Lọc các giá trị khác 0
df3_filtered = df3[df3['Close'] != 0]

# Plotting
trace1 = go.Scatter(x=df3_filtered['Date'], y=df3_filtered['Close'], name='Giá', line=dict(color='blue', width=2))
trace3 = go.Scatter(x=df3_filtered['Date'], y=df3_filtered['MA_50'], name='50-day MA', line=dict(color='green', width=2))
trace4 = go.Scatter(x=df3_filtered['Date'], y=df3_filtered['MA_200'], name='200-day MA', line=dict(color='Orange', width=2))

layout = go.Layout(title='Đồ thị giá')

figure1 = go.Figure(data=[trace1, trace3, trace4], layout=layout)

#Vẽ đường bollinger bands
window = 20
df3['Bollinger_Middle'] = df3['Close'].rolling(window=window).mean()
df3['Bollinger_Std'] = df3['Close'].rolling(window=window).std()
df3['Bollinger_Upper'] = df3['Bollinger_Middle'] + 2 * df3['Bollinger_Std']
df3['Bollinger_Lower'] = df3['Bollinger_Middle'] - 2 * df3['Bollinger_Std']

# Lọc dữ liệu để loại bỏ các hàng có giá trị 0
df3_filtered = df3[df3['Close'] != 0]

# Tạo đối tượng đồ thị
trace1 = go.Scatter(x=df3_filtered['Date'], y=df3_filtered['Close'], name='Close', line=dict(color='blue',width=1.5))
trace2 = go.Scatter(x=df3_filtered['Date'], y=df3_filtered['Bollinger_Upper'], name='Bollinger Upper', line=dict(color='Orange', width=2.5))
trace3 = go.Scatter(x=df3_filtered['Date'], y=df3_filtered['Bollinger_Lower'], name='Bollinger Lower', line=dict(color='green', width=2))

# Tạo layout với màu nền là 'lightgrey'
layout = go.Layout(title='Bollinger Bands', plot_bgcolor='darkgrey')

# Tạo đối tượng đồ thị tổng
figure2 = go.Figure(data=[trace1, trace2, trace3], layout=layout)

#MACD
# Set short and long window parameters
short_window = 12
long_window = 26

# Calculate moving averages
df3['Short_MA'] = df3['Close'].rolling(window=short_window).mean()
df3['Long_MA'] = df3['Close'].rolling(window=long_window).mean()
df3['macd'] = df3['Short_MA'] - df3['Long_MA']
df3['signal_line'] = df3['macd'].rolling(window=9).mean()
df3['macd_histogram'] = df3['macd'] - df3['signal_line']

# Create MACD plot using Plotly Express
figure3 = px.line(df3, x=df3['Date'], y=['macd', 'signal_line'], labels={'value': 'MACD/Signal Line', 'Date': 'Date'},
              title='MACD and Signal Line')
figure3.add_bar(x=df3['Date'], y=df3['macd_histogram'], name='Histogram', opacity=0.9, marker=dict(color='blue'))

# Set axis labels
figure3.update_xaxes(title_text='Date')
figure3.update_yaxes(title_text='MACD/Signal Line', secondary_y=False)
figure3.update_yaxes(title_text='Histogram', secondary_y=True)
# Tạo layout với màu nền là 'lightgrey'
layout = go.Layout(title='MACD and Signal Line', plot_bgcolor='darkgrey')

#VẼ BIỂU ĐỒ RSSI
# Tính toán RSI
df3['Price Change'] = df3['Close'].diff()
df3['Gain'] = df3['Price Change'].where(df3['Price Change'] > 0, 0)
df3['Loss'] = -df3['Price Change'].where(df3['Price Change'] < 0, 0)
average_gain = df3['Gain'].rolling(window=14).mean()
average_loss = df3['Loss'].rolling(window=14).mean()
relative_strength = average_gain / average_loss
rsi = 100 - (100 / (1 + relative_strength))
df3['RSI'] = rsi

# Chuyển dữ liệu từ wide-form sang long-form, giữ lại cột 'Date' và 'Close'
df3_long = df3.melt(id_vars=['Date', 'Close'], value_vars=['RSI'],
                    var_name='Indicator', value_name='Value')


# Tạo biểu đồ RSI bằng Plotly Express
figure4 = px.line(df3_long, x="Date", y="Value", color="Indicator",
              labels={"Date": "Date", "Value": "Price"},
              title="[BCM] Relative Strength Index (RSI)")
figure4.add_shape(
    type='line',
    x0=df3_long['Date'].iloc[13],  # Chỉ số 14 - 1 (vì window=14)
    x1=df3_long['Date'].iloc[-1],
    y0=70,
    y1=70,
    line=dict(color='red', dash='dash'),
    xref='x',
    yref='y'
)
figure4.add_shape(
    type='line',
    x0=df3_long['Date'].iloc[13],  # Chỉ số 14 - 1 (vì window=14)
    x1=df3_long['Date'].iloc[-1],
    y0=30,
    y1=30,
    line=dict(color='green', dash='dash'),
    xref='x',
    yref='y'
)
#BIỂU ĐỒ GIÁ VỚI KHỐI LƯỢNG
close_trace = go.Scatter(x=vl4['Date'], y=vl4['Close'], mode='lines', name='Close', marker=dict(color='blue'))
# Lọc dữ liệu để loại bỏ các hàng có giá trị 0
df3_filtered = df3[df3['Close'] != 0]
# Create a bar trace for volume
volume_bar = go.Bar(x=vl4['Date'], y=vl4['Vol'], name='Vol', marker=dict(color='green'))

# Create the layout
layout = go.Layout(title='Closing Prices with Volume',
                   xaxis=dict(rangeslider=dict(visible=False)),
                   yaxis=dict(title='Close'),
                   yaxis2=dict(title='Vol', overlaying='y', side='right'))

# Create a figure with close and volume traces
figure5 = go.Figure(data=[close_trace, volume_bar], layout=layout)

#CÀI ĐẶT LAYOUT CHO DASH
# Danh sách các biểu đồ
financial_chart = ['Tăng trưởng doanh thu',
                   'Doanh thu - Lợi nhuận gộp - LNST', 
                   'Cơ cấu Tài sản',
                   'Cơ cấu Tài sản Dài hạn' ,
                   'Cơ cấu Tài sản Ngắn hạn', 
                   'Biên lợi nhuận theo năm',
                   'Khả năng sinh lời',
                   'Dòng tiền',
                   'Cấu trúc Nguồn Vốn']
                                        
stock_charts = ['Đồ thị Giá (Stock Price)', 
                'Bollinger Bands',
                'MACD và Đường Signal',
                'RSI',
                'Closing Prices with Volume']

# Layout cho phần tìm kiếm và hiển thị biểu đồ tài chính
financial_layout = html.Div([
    html.Div([
        dcc.Dropdown(
            id='financial-chart-dropdown',
            options=[{'label': chart, 'value': chart} for chart in financial_chart],
            value=financial_chart[0],
            style={'width': '90%', 'margin-top': '10px'}
        ),
    ], style={'width': '50%'}),
    html.Div([
        dcc.Loading(
            id="financial-loading-chart",
            type="default",
            children=[
                dcc.Graph(id='financial-selected-chart', style={'width': '100%', 'height': '500px'}),
            ],
        ),
    ], style={'width': '100%', 'margin-top': '20px'}),
], style={'display': 'flex', 'flex-direction': 'column', 'justify-content': 'center', 'align-items': 'center'})

# Layout cho phần tìm kiếm và hiển thị biểu đồ cổ phiếu
stock_layout = html.Div([
    html.Div([
        dcc.Dropdown(
            id='stock-chart-dropdown',
            options=[{'label': stock, 'value': stock} for stock in stock_charts],
            value=stock_charts[0],
            style={'width': '90%', 'margin-top': '-20px'}
        ),
    ], style={'width': '50%', 'float': 'left'}),
    html.Div([
        dcc.Loading(
            id="stock-loading-chart",
            type="default",
            children=[
                dcc.Graph(id='stock-selected-chart', style={'width': '100%', 'height': '500px'})
            ]
        ),
    ], style={'width': '100%', 'margin-top': '20px'}),
], style={'display': 'flex', 'flex-direction': 'column', 'justify-content': 'center', 'align-items': 'center'})



# Tạo layout chính
app.layout = html.Div([
    html.Div([
        html.H1("PHÂN TÍCH TÀI CHÍNH GIAI ĐOẠN 2018-2022 CỦA BCM", style={'text-align': 'center', 'margin': 'auto'}),
    ], style={'color': 'red', 'display': 'flex', 'align-items': 'center', 'text-align': 'center'}),

    # Thêm thông tin công ty
    html.Div([
        html.H3("Sơ lược về công ty Becamex IDC:", style={'color': 'blue', 'text-align': 'center', 'fontSize': 30}),
        html.P("Becamex IDC Corp (Với mã chứng khoán là BCM), được thành lập từ năm 1976, sau hơn 40 năm hoạt động đến nay BCM đã trở thành động lực hàng đầu trong lĩnh vực đầu tư và phát triển công nghiệp tại Việt Nam. Với trụ sở tại Bình Dương, BCM hoạt động ở đa ngành nghề trong các lĩnh vực đa dạng như chứng khoán, tài chính, bảo hiểm, xây dựng, thương mại, và nhiều lĩnh vực khác. Becamex IDC hiện có 28 công ty thành viên, đầu tư và phát triển các dự án theo mô hình BOT, điều này đã làm nổi bật tầm ảnh hưởng và uy tín của BCM. Với vốn điều lệ 8.500 tỷ đồng và tổng tài sản hơn 100.000 tỷ đồng, Becamex IDC không chỉ là doanh nghiệp hàng đầu mà còn tiên phong trong sáng tạo và phát triển công nghiệp Việt Nam."),
    ], style={'background-color': 'orange', 'color': 'black', 'padding': '10px', 'border-radius': '15px', 'margin-bottom': '10px', 'fontSize': 20}),

    # Thêm thông tin chi tiết
    html.Div([
        html.H4("Thông tin chi tiết:", style={'text-align': 'center', 'color': 'green','fontSize': 30}),
        html.P("Dưới đây là bảng chi tiết về các thông số tài chính của công ty.",style={'fontSize': 20}),
    ], style={'margin-bottom': '10px'}),

    # Thêm bảng dữ liệu
    DataTable(
        id='datatable',
        columns=[{'name': col, 'id': col} for col in df_info.columns],
        data=df_info.to_dict('records'),
        style_table={'overflowX': 'auto', 'margin': 'auto'},
        style_cell={
            'textAlign': 'center',
            'fontFamily': 'Times New Roman',
        },
    ),
  
    # Thêm dòng chữ trước khi chia layout thành hai phần chính
    html.Div([
        html.P("Dưới đây là một số biểu đồ phân tích tình hình tài chính giai đoạn 2018-2022 của BCM. Gồm các biểu đồ phân tích cổ phiếu và các biểu đồ phân tích tài chính được vẽ từ báo cáo tài chính của công ty này.",
               style={ 'background-color': 'green','color': 'white', 'fontSize': 19}),
    ], style={'margin-bottom': '20px'}),

    # Chia layout thành hai phần (financial và stock)
    html.Div([
        html.Div([
            html.Div(financial_layout, className='col-md-6'),  # financial_layout chiếm 50% chiều rộng
            html.Div(stock_layout, className='col-md-6'),  # stock_layout chiếm 50% chiều rộng
        ], className='row'),
    ], className='container')

])

# Cập nhật biểu đồ tài chính dựa trên mục được chọn từ Dropdown
@app.callback(
    Output('financial-selected-chart', 'figure'),
    [Input('financial-chart-dropdown', 'value')]
)
def update__financial_chart(selected_chart):
    # Thực hiện xử lý để cập nhật biểu đồ tài chính
    if selected_chart == 'Tăng trưởng doanh thu':
        return fig1
    elif selected_chart == 'Doanh thu - Lợi nhuận gộp - LNST':
        return fig2
    elif selected_chart == 'Cơ cấu Tài sản':
        return fig3
    elif selected_chart == 'Cơ cấu Tài sản Dài hạn':
        return fig4
    elif selected_chart == 'Cơ cấu Tài sản Ngắn hạn':              
        return fig5
    elif selected_chart == 'Biên lợi nhuận theo năm':
        return fig6
    elif selected_chart =='Khả năng sinh lời':
        return fig7
    elif selected_chart =='Dòng tiền':
        return fig8
    elif selected_chart == 'Cấu trúc Nguồn Vốn':
        return fig10
    else:
        return go.Figure()

# Cập nhật biểu đồ cổ phiếu dựa trên mục được chọn từ Dropdown
@app.callback(
    Output('stock-selected-chart', 'figure'),
    [Input('stock-chart-dropdown', 'value')]
)
def update_stock_charts(selected_chart):
    # Thực hiện xử lý để cập nhật biểu đồ cổ phiếu
    if selected_chart == 'Đồ thị Giá (Stock Price)':
        return figure1
    if selected_chart == 'Bollinger Bands':
        return figure2
    if selected_chart == 'MACD và Đường Signal':
        return figure3
    if selected_chart == 'RSI':
        return figure4
    if selected_chart == 'Closing Prices with Volume':
        return figure5
    else:
        return go.Figure()

# Chạy ứng dụng Dash
if __name__ == '__main__':
    app.run_server(debug=True, port=8057)

    # Mở trình duyệt với đường link
    webbrowser.open_new("http://localhost:8057")


d:\VSCODE\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

d:\VSCODE\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

d:\VSCODE\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

d:\VSCODE\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarni